In [1]:
import os
from autovc.speaker_encoder.utils import *
import soundfile as sf
from autovc.utils.hparams import SpeakerEncoderParams as params
from autovc.utils.preprocess_wav import audio_to_melspectrogram
import numpy as np

In [1]:
# get data
# import wandb
# run = wandb.init()
# artifact = run.use_artifact('deep_voice_inc/data/smk_speakers:v0', type='dataset')
# artifact_dir = artifact.download(root="data")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: s183920 (use `wandb login --relogin` to force relogin)
C:\Users\lukas\miniconda3\envs\AutoVC-env\lib\site-packages\IPython\html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


wandb: Downloading large artifact smk_speakers:v0, 4947.34MB. 3032 files... Done. 0:0:0


In [ ]:
walk = [w for w in os.walk("data/yang_test")][0]
root, dirs, files = [w for w in walk]

for file in files:
    waveform = preprocess_wav(os.path.join(root, file))
    sf.write(f"test_yang_silence/{file}", np.asarray(waveform), samplerate = 16000)

In [3]:
walk = [w for w in os.walk("test_yang_silence")][0]
_, _, files = [w for w in walk]

specs_org = []
specs_new = []

for file in files:
    specs_org.append(audio_to_melspectrogram(os.path.join("data/yang_test", file)))
    specs_new.append(audio_to_melspectrogram(os.path.join("test_yang_silence", file)))



In [13]:
lens_new = np.array([np.shape(s)[1] for s in specs_new])
lens_org = np.array([np.shape(s)[1] for s in specs_org])


# (lens_org-lens_new)/lens_org
lens_new.std()
lens_new.mean()


186.2

In [2]:
hparams = params()

def create_audio_mask(wav):
    """"
    Creates an audio mask, where False indicates silence.

    :param wav: a numpy array with the content of a wav file
    :return audio_mask: the calculated audio mask
    """
    # Compute the voice detection window size
    samples_per_window = (hparams.vad_window_length * hparams.sampling_rate) // 1000
    
    # Trim the end of the audio to have a multiple of the window size
    wav = wav[:len(wav) - (len(wav) % samples_per_window)]
    
    # Convert the float waveform to 16-bit mono PCM
    pcm_wave = struct.pack("%dh" % len(wav), *(np.round(wav * int16_max)).astype(np.int16))
    
    # Perform voice activation detection
    voice_flags = []
    vad = webrtcvad.Vad(mode=3)
    for window_start in range(0, len(wav), samples_per_window):
        window_end = window_start + samples_per_window
        voice_flags.append(vad.is_speech(pcm_wave[window_start * 2:window_end * 2],
                                         sample_rate=hparams.sampling_rate))
    voice_flags = np.array(voice_flags)
    
    # Smooth the voice detection with a moving average
    def moving_average(array, width):
        array_padded = np.concatenate((np.zeros((width - 1) // 2), array, np.zeros(width // 2)))
        ret = np.cumsum(array_padded, dtype=float)
        ret[width:] = ret[width:] - ret[:-width]
        return ret[width - 1:] / width
    
    audio_mask = moving_average(voice_flags, hparams.vad_moving_average_width)
    audio_mask = np.round(audio_mask).astype(bool)
    
    # Dilate the voiced regions
    audio_mask = binary_dilation(audio_mask, np.ones(hparams.vad_max_silence_length + 1))
    audio_mask = np.repeat(audio_mask, samples_per_window)

    return audio_mask

def split_audio(wav_path, save_folder = None, allowed_pause = .5):
    """
    Chops the content of the wav file into multiple files, based on when there is
    a longer period if silence. Files will be saved with a number indicating the order the content appeared in.

    :param wav: path to wav file
    :param save_folder: folder to save files in
    :allowed_pause: number seconds of silence to allow in a sound file
    :return: content of the wav file seperated in multiple files 
    """
    # load wav
    wav, source_sr = librosa.load(wav_path, sr=None)

    audio_mask = create_audio_mask(wav)

    # function for finding consecutive values without silence
    def consecutive(data, stepsize=1):
        return np.split(data, np.where(np.diff(data) != stepsize)[0]+1)


    allowed_pause = allowed_pause*source_sr
    for i, split in enumerate(consecutive(np.where(audio_mask)[0])):
        if i == 0:
            joined_splits = [split]
        else:
            # join last subset with new split if difference is less than allowed pause
            if split[-1] - joined_splits[-1][-1] <= allowed_pause:
                joined_splits.append(np.concatenate([joined_splits.pop(), split]))
            else:
                joined_splits.append(split)
    
    # save chopped files
    filename = os.path.split(wav_path)[-1]
    wav_splitted = []
    if save_folder is not None:
        os.makedirs(save_folder, exist_ok=True)
        
        for i, split in enumerate(joined_splits):
            fname = filename.replace(".wav", f"_{str(i+1).zfill(3)}.wav")
            wav_splitted.append(wav[split])
            sf.write(f"{save_folder}/{fname}", wav_splitted[-1], samplerate = source_sr)


    return wav_splitted




split_audio("data/samples/hilde_677.wav", "results")

NameError: name 'params' is not defined

In [56]:
# a = np.array([7040,  7041,  7042, 89597, 89598, 89599])
a = np.array([*[False]*4, *[True]*3, False, *[True]*4])
# a = np.diff(a)
# np.where((~a[:-1])*np.roll(a, 1)[1:])
a
np.where((a[:-1]==(np.roll(a, 1)[1:])))
# np.array_split(wav, mask)
# a[:-1]*(np.roll(a, 1)[1:] == False)
a[:-1]!=np.roll(a, 2)[1:]
# np.roll(a, 2)[1:]

array([ True, False, False, False,  True, False, False,  True,  True,
       False, False])

In [110]:
print(mins, maxs)

[ 7040 50880 72960 84160] [39359 63039 82239 89599]


In [145]:
def consecutive(data, stepsize=1):
    return np.split(data, np.where(np.diff(data) != stepsize)[0]+1)

cons = consecutive(np.where(mask)[0])
mins = np.array([arr.min() for arr in cons])
maxs = np.array([arr.max() for arr in cons])




# len(wav)/22050
# source_sr
# import copy
# cons2 = copy.copy(cons)


allowed_pause = .4*22050
joined_cons = [cons[0]]
for con in cons[1:]:
    # print(joined_cons)
    if con[-1] - joined_cons[-1][-1] <= allowed_pause:
        
        prev = joined_cons.pop()
        print(prev)
        joined_cons.append(np.concatenate([prev, con]))
    else:
        joined_cons.append(con)

joined_cons


# consecutive(np.where((mins[1:]-maxs[:-1]) < .5*22050)[0])



[72960 72961 72962 ... 82237 82238 82239]


[array([ 7040,  7041,  7042, ..., 39357, 39358, 39359], dtype=int64),
 array([50880, 50881, 50882, ..., 63037, 63038, 63039], dtype=int64),
 array([72960, 72961, 72962, ..., 89597, 89598, 89599], dtype=int64)]

In [126]:


print(maxs, mins) 

[39359 63039 82239 89599] [ 7040 50880 72960 84160]


In [137]:
a = (mins[1:]-maxs[:-1]) < .5*22050
consecutive(np.where(a)[0])


[array([1, 2], dtype=int64)]

In [5]:
from autovc.speaker_encoder.utils import split_audio
split_audio("data/samples/hilde_677.wav", "results", allowed_pause=2)

[array([ 0.01443481,  0.01394653,  0.0138855 , ..., -0.01269531,
        -0.01290894, -0.01272583], dtype=float32),
 array([ 0.00180054,  0.00198364,  0.00192261, ..., -0.00192261,
        -0.00192261, -0.00192261], dtype=float32),
 array([-0.00204468, -0.00213623, -0.00216675, ..., -0.0005188 ,
        -0.00109863, -0.00177002], dtype=float32)]

In [4]:
import wandb
run = wandb.init(entity = "deep_voice_inc")
# artifact = run.use_artifact('deep_voice_inc/SpeakerEncoder/model_20220113.pt:v4', type='AutoEncoder')
# artifact_dir = artifact.download()
# artifact_dir
run.config["tester"] = 10

In [1]:
import torch
torch.load("models/AutoVC/wandb/model_20220113.pt")

{'step': 71,
 'model_state': OrderedDict([('encoder.convolutions.0.0.conv.weight',
               tensor([[[-0.0776, -0.0904, -0.0703, -0.0715,  0.0059],
                        [-0.1031, -0.0656, -0.0931, -0.0412,  0.0877],
                        [-0.0205, -0.1249, -0.0503,  0.0577,  0.1640],
                        ...,
                        [-0.0626, -0.0563, -0.0158, -0.0524, -0.0244],
                        [ 0.1648,  0.1877,  0.1346,  0.1943,  0.1588],
                        [ 0.0098, -0.0401, -0.0304, -0.0030,  0.0223]],
               
                       [[-0.0258,  0.0080, -0.0572, -0.0728, -0.0191],
                        [ 0.0568, -0.0166, -0.0063, -0.0970, -0.0458],
                        [ 0.0277, -0.0460, -0.0345, -0.1310, -0.0932],
                        ...,
                        [-0.0519,  0.0051,  0.0472,  0.0149, -0.0595],
                        [ 0.1153,  0.0157,  0.0616, -0.0022, -0.0176],
                        [ 0.0173,  0.0032,  0.0095,  0.0144, 